<a href="https://colab.research.google.com/github/tonyw54/GeorgeBrown/blob/main/Computer_Vision2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
import zipfile
import shutil
import os

# Re-download competition data
computer_vision_xm_path = kagglehub.competition_download('computer-vision-xm')

source_dir = '/root/.cache/kagglehub/competitions/computer-vision-xm'
destination_dir = '/content/computer-vision-xm'  # Create a folder in '/content/'

shutil.copytree(source_dir, destination_dir)

shutil.rmtree('/root/.cache/kagglehub/competitions/computer-vision-xm', ignore_errors=True)

print(f"Directory '{source_dir}' copied to '{destination_dir}' successfully.")

100%|██████████| 6.20G/6.20G [03:37<00:00, 30.7MB/s]

Extracting files...


Directory '/root/.cache/kagglehub/competitions/computer-vision-xm' copied to '/content/computer-vision-xm' successfully.


# 1. Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
import os
import shutil
from PIL import Image
from tqdm import tqdm
import random
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
# Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    print("Cuda is up")
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

Cuda is up


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df.iloc[idx, 1]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path)
        label = self.labels_df.iloc[idx, 2] if 'Labels' in self.labels_df.columns else -1

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.dropout1 = nn.Dropout(p=0.1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.dropout2 = nn.Dropout(p=0.15)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.dropout3 = nn.Dropout(p=0.2)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.dropout4 = nn.Dropout(p=0.25)
        self.pool = nn.MaxPool2d(2, 2)
        # Instead of initializing with 0 input features, we calculate the size after a dummy forward pass
        dummy_input = torch.randn(1, 3, 400, 600) # Create a dummy input tensor with the expected size
        dummy_output = self.feature_extractor(dummy_input)
        fc1_input_size = dummy_output.view(dummy_output.size(0), -1).shape[1]
        self.fc1 = nn.Linear(fc1_input_size, 512)
        self.dropout = nn.Dropout(p=0.5)  # Add dropout with a probability of 0.5
        self.fc2 = nn.Linear(512, num_classes)


    def feature_extractor(self, x):
        """Helper function to extract features before the fully connected layers."""
        x = F.relu(self.conv1(x))
        x = self.dropout1(x)
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.dropout2(x)
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.dropout3(x)
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.dropout4(x)
        x = self.pool(x)
        return x


    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()
print(model)

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=236800, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
# Define separate transforms for training and testing
train_transform = transforms.Compose([
    transforms.Resize((400, 600)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),  # Add rotation
    transforms.RandomResizedCrop(size=(400, 600), scale=(0.8, 1.0)),  # Add random cropping
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10), # Add random affine transformations
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Add color jittering
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


test_transform = transforms.Compose([
    transforms.Resize((400, 600)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create datasets with respective transforms
train_dataset = CustomDataset(csv_file='/content/computer-vision-xm/train.csv', img_dir='/content/computer-vision-xm/images/kaggle/working/Reorganized_Data/images', transform=train_transform)
test_dataset = CustomDataset(csv_file='/content/computer-vision-xm/test.csv', img_dir='/content/computer-vision-xm/images/kaggle/working/Reorganized_Data/images', transform=test_transform)

# Split the training dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, prefetch_factor=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, prefetch_factor=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, prefetch_factor=2, pin_memory=True)

In [ ]:
# Check sizes
for images, labels in train_loader:
    print("Image shape:", images.shape)  # Should be (batch_size, channels, height, width)
    print("Label shape:", labels.shape)  # Should be (batch_size)

    # Forward pass
    outputs = model(images)
    print("Output shape:", outputs.shape)  # Should be (batch_size, 2) for binary classification
    break

Image shape: torch.Size([32, 3, 400, 600])
Label shape: torch.Size([32])
Output shape: torch.Size([32, 2])


In [ ]:
# Calculate the total number of parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {total_params}')

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.001)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

Total number of parameters: 121631554
cuda


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=236800, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [ ]:
# Training loop with early stopping
epochs = 30
patience = 10
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc="Training"):
        images = images.to(device)
        labels = labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    print(f'Training loss: {running_loss / len(train_loader)}')

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation"):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Convert the outputs to predicted class labels
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f'Validation loss: {val_loss / len(val_loader)}')
    print(f'Validation accuracy: {val_accuracy:.4f}')

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'Best model saved with valid loss of {val_loss / len(val_loader)}')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

model.save_state_dict(torch.load('best_model.pth'))

Epoch 1/30


Training: 100%|██████████| 85/85 [05:07<00:00,  3.62s/it]


Training loss: 0.6798048503258649


Validation: 100%|██████████| 22/22 [01:19<00:00,  3.63s/it]


Validation loss: 0.6437617621638558
Validation accuracy: 0.6755
Best model saved with valid loss of 0.6437617621638558
Epoch 2/30


...

Training: 100%|██████████| 85/85 [05:02<00:00,  3.56s/it]


Training loss: 0.30335085926687017


Validation: 100%|██████████| 22/22 [01:18<00:00,  3.55s/it]


Validation loss: 0.3097642186013135
Validation accuracy: 0.8761
Epoch 9/30


Training:  47%|████▋     | 40/85 [02:24<02:15,  3.01s/it]

In [ ]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))
model.to(device)

# Make predictions on the test dataset
model.eval()
predictions = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Create a DataFrame with IDs and predictions
submission_df = pd.DataFrame({
    'Images': test_dataset.labels_df['Images'],
    'Labels': predictions
})
submission_df.to_csv('submission.csv', index=False)
print('Submission file was created.')

NameError: name 'model' is not defined